In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [2]:
def read_data(farms_csv, processing_csv, centers_csv):
    """Reads CSVs and returns data dictionaries, handling string IDs"""
    farms_df = pd.read_csv(farms_csv)
    farms_df["Farm_ID"] = farms_df["Farm_ID"].str.replace("Farm_", "").astype(int)
    
    plant_df = pd.read_csv(processing_csv)
    plant_df["Processing_Plant_ID"] = plant_df["Processing_Plant_ID"].str.replace("Plant_", "").astype(int)

    centers_df = pd.read_csv(centers_csv)
    centers_df["Center_ID"] = centers_df["Center_ID"].str.replace("Center_", "").astype(int)

    farm_ids = farms_df["Farm_ID"].unique().tolist()
    farm_ids.sort()
    plant_ids = plant_df["Processing_Plant_ID"].unique().tolist()
    plant_ids.sort()
    center_ids = centers_df["Center_ID"].unique().tolist()
    center_ids.sort()

    farm_capacity = dict(zip(farms_df["Farm_ID"], farms_df["Bio_Material_Capacity_Tons"]))
    farm_raw_cost = dict(zip(farms_df["Farm_ID"], farms_df["Cost_Per_Ton"]))

    trans_f2p_cols = [c for c in farms_df.columns if c.startswith("Transport_Cost_To_Plant_")]
    farm_to_plant_cost = {}
    for _, row in farms_df.iterrows():
        f_id = row["Farm_ID"]
        for col in trans_f2p_cols:
            p_id_str = col.split("Transport_Cost_To_Plant_")[1]
            p_id = int(p_id_str)
            cost_val = row[col]
            farm_to_plant_cost[(f_id, p_id)] = cost_val

    plant_region = dict(zip(plant_df["Processing_Plant_ID"], plant_df["Region"]))
    plant_capacity = dict(zip(plant_df["Processing_Plant_ID"], plant_df["Capacity_Tons"]))
    plant_proc_cost = dict(zip(plant_df["Processing_Plant_ID"], plant_df["Processing_Cost_Per_Ton"]))

    trans_p2c_cols = [c for c in plant_df.columns if c.startswith("Transport_Cost_To_Center_")]
    plant_to_center_cost = {}
    for _, row in plant_df.iterrows():
        p_id = row["Processing_Plant_ID"]
        for col in trans_p2c_cols:
            c_id_str = col.split("Transport_Cost_To_Center_")[1]
            c_id = int(c_id_str)
            cost_val = row[col]
            plant_to_center_cost[(p_id, c_id)] = cost_val

    center_demand = dict(zip(centers_df["Center_ID"], centers_df["Requested_Demand_Tons"]))
    center_region = dict(zip(centers_df["Center_ID"], centers_df["Region"]))

    return farm_ids, plant_ids, center_ids, farm_capacity, farm_raw_cost, farm_to_plant_cost, plant_capacity, plant_proc_cost, plant_to_center_cost, center_demand, center_region, plant_region

In [3]:
def solve_bioagri(farms_csv, processing_csv, centers_csv):
    """Base model with 3 stage supply chain"""
    farm_ids, plant_ids, center_ids, farm_capacity, farm_raw_cost, farm_to_plant_cost, plant_capacity, plant_proc_cost, plant_to_center_cost, center_demand, _, _ = read_data(farms_csv, processing_csv, centers_csv)

    model = gp.Model("BioAgri_FixedIDs")
    x = {}
    for f_id in farm_ids:
        for p_id in plant_ids:
            x[(f_id, p_id)] = model.addVar(lb=0.0, name=f"x_{f_id}_{p_id}")
    y = {}
    for p_id in plant_ids:
        for c_id in center_ids:
            y[(p_id, c_id)] = model.addVar(lb=0.0, name=f"y_{p_id}_{c_id}")

    farm_transport_expr = gp.quicksum(
        (farm_raw_cost[f_id] + farm_to_plant_cost[(f_id, p_id)]) * x[(f_id, p_id)]
        for f_id in farm_ids
        for p_id in plant_ids
    )
    proc_expr = gp.quicksum(
        plant_proc_cost[p_id] * gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids)
        for p_id in plant_ids
    )
    dist_expr = gp.quicksum(
        plant_to_center_cost[(p_id, c_id)] * y[(p_id, c_id)]
        for p_id in plant_ids
        for c_id in center_ids
    )
    model.setObjective(farm_transport_expr + proc_expr + dist_expr, GRB.MINIMIZE)

    for f_id in farm_ids:
        model.addConstr(
            gp.quicksum(x[(f_id, p_id)] for p_id in plant_ids) <= farm_capacity[f_id],
            name=f"Max_Farm_Capacity_Farm_{f_id}"
        )

    for p_id in plant_ids:
        model.addConstr(
            gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids) <= plant_capacity[p_id],
            name=f"Max_Plant_Capacity_Plant_{p_id}"
        )
    for p_id in plant_ids:
        model.addConstr(
            gp.quicksum(y[(p_id, c_id)] for c_id in center_ids) ==
            gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids),
            name=f"Flow_Conservation_Plant_{p_id}"
        )
    for c_id in center_ids:
        model.addConstr(
            gp.quicksum(y[(p_id, c_id)] for p_id in plant_ids) == center_demand[c_id],
            name=f"CenterDem_{c_id}"
        )
    model.optimize()
    if model.status == GRB.OPTIMAL:
        print(f"\nOptimal solution found. Minimum Cost = {model.objVal:,.2f}\n")
        farm_plant_data = []
        for (f_id, p_id) in x:
          if x[(f_id, p_id)].X > 1e-6:
             farm_plant_data.append({"Farm": f_id, "Plant": p_id, "Flow (tons)": x[(f_id, p_id)].X})
        
        farm_plant_df = pd.DataFrame(farm_plant_data)
        print("\\nFarm->Plant Flows:\\n", farm_plant_df)

        plant_center_data = []
        for (p_id, c_id) in y:
            if y[(p_id, c_id)].X > 1e-6:
              plant_center_data.append({"Plant": p_id, "Center": c_id, "Flow (tons)": y[(p_id, c_id)].X})

        plant_center_df = pd.DataFrame(plant_center_data)
        print("\\nPlant->Center Flows:\\n", plant_center_df)
    else:
        print(f"Model ended with status {model.status}")

    print("Objective of Base Model: \n", model.getObjective())
    print("\\nConstraints of Base Model:\\n", model.getConstrs())

    return model


In [4]:
def solve_bioagri_same_region(farms_csv, processing_csv, centers_csv):
    """Same as base but only allows transport between plants and centers of same region."""
    farm_ids, plant_ids, center_ids, farm_capacity, farm_raw_cost, farm_to_plant_cost, plant_capacity, plant_proc_cost, plant_to_center_cost, center_demand, center_region, plant_region = read_data(farms_csv, processing_csv, centers_csv)

    model = gp.Model("BioAgri_SameRegion")
    x = {}
    for f_id in farm_ids:
        for p_id in plant_ids:
            x[(f_id, p_id)] = model.addVar(lb=0.0, name=f"x_{f_id}_{p_id}")
    y = {}
    for p_id in plant_ids:
        for c_id in center_ids:
            if (p_id, c_id) in plant_to_center_cost:
                y[(p_id, c_id)] = model.addVar(lb=0.0, name=f"y_{p_id}_{c_id}")
                print(f"Variable Created {p_id} to {c_id}")

    farm_transport_expr = gp.quicksum(
        (farm_raw_cost[f_id] + farm_to_plant_cost[(f_id, p_id)]) * x[(f_id, p_id)]
        for f_id in farm_ids
        for p_id in plant_ids
    )
    proc_expr = gp.quicksum(
        plant_proc_cost[p_id] * gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids)
        for p_id in plant_ids
    )
    dist_expr = gp.quicksum(
        plant_to_center_cost[(p_id, c_id)] * y[(p_id, c_id)]
        for (p_id, c_id) in plant_to_center_cost
    )
    model.setObjective(farm_transport_expr + proc_expr + dist_expr, GRB.MINIMIZE)

    for f_id in farm_ids:
        model.addConstr(
            gp.quicksum(x[(f_id, p_id)] for p_id in plant_ids) <= farm_capacity[f_id],
            name=f"Max_Farm_Capacity_Farm_{f_id}"
        )

    for p_id in plant_ids:
        model.addConstr(
            gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids) <= plant_capacity[p_id],
            name=f"Max_Plant_Capacity_Plant_{p_id}"
        )
    for p_id in plant_ids:
        in_flow = gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids)
        out_flow = gp.quicksum(y[(p_id, c_id)] for c_id in center_ids if (p_id, c_id) in y)
        model.addConstr(out_flow == in_flow, name=f"Flow_Conservation_Plant_{p_id}")
    for c_id in center_ids:
        inbound = gp.quicksum(y[(p_id, c_id)] for p_id in plant_ids if (p_id, c_id) in y)
        model.addConstr(inbound == center_demand[c_id], name=f"CenterDem_{c_id}")
        
    model.optimize()
    if model.status == GRB.OPTIMAL:
        print(f"\nOptimal solution found (same-region only). Minimum Cost = {model.objVal:,.2f}\n")
        
        farm_plant_data = []
        for (f_id, p_id) in x:
          if x[(f_id, p_id)].X > 1e-6:
             farm_plant_data.append({"Farm": f_id, "Plant": p_id, "Flow (tons)": x[(f_id, p_id)].X, "Plant Region": plant_region[p_id]})
        
        farm_plant_df = pd.DataFrame(farm_plant_data)
        print("\\nFarm->Plant Flows:\\n", farm_plant_df)

        plant_center_data = []
        for (p_id, c_id) in y:
            if y[(p_id, c_id)].X > 1e-6:
              plant_center_data.append({"Plant": p_id, "Center": c_id, "Flow (tons)": y[(p_id, c_id)].X, "Center Region": center_region[c_id]})
        plant_center_df = pd.DataFrame(plant_center_data)
        print("\\nPlant->Center Flows:\\n", plant_center_df)
    else:
        print(f"Model ended with status {model.status}")

    print("Objective of Same Region Model: \n", model.getObjective())
    print("\\nConstraints of Same Region Model:\\n", model.getConstrs())
    
    return model

In [5]:
def solve_bioagri_high_quality(farms_csv, processing_csv, centers_csv):
    """Only allows raw material from farms with a quality of level 3 or 4"""
    farm_ids, plant_ids, center_ids, farm_capacity, farm_raw_cost, farm_to_plant_cost, plant_capacity, plant_proc_cost, plant_to_center_cost, center_demand, _, _ = read_data(farms_csv, processing_csv, centers_csv)

    farms_df = pd.read_csv(farms_csv)
    farms_df["Farm_ID"] = farms_df["Farm_ID"].str.replace("Farm_", "").astype(int)
    farms_df = farms_df[ farms_df["Quality"] >= 3 ].copy()
    farm_ids = farms_df["Farm_ID"].unique().tolist()
    farm_ids.sort()
    farm_capacity = dict(zip(farms_df["Farm_ID"], farms_df["Bio_Material_Capacity_Tons"]))
    farm_raw_cost = dict(zip(farms_df["Farm_ID"], farms_df["Cost_Per_Ton"]))

    trans_f2p_cols = [c for c in farms_df.columns if c.startswith("Transport_Cost_To_Plant_")]
    farm_to_plant_cost = {}
    for _, row in farms_df.iterrows():
        f_id = row["Farm_ID"]
        for col in trans_f2p_cols:
            p_id_str = col.split("Transport_Cost_To_Plant_")[1]
            p_id = int(p_id_str)
            cost_val = row[col]
            farm_to_plant_cost[(f_id, p_id)] = cost_val
    
    model = gp.Model("BioAgri_HighQualityOnly")
    x = {}
    for f_id in farm_ids:
        for p_id in plant_ids:
            x[(f_id, p_id)] = model.addVar(lb=0.0, name=f"x_{f_id}_{p_id}")
    y = {}
    for p_id in plant_ids:
        for c_id in center_ids:
            y[(p_id, c_id)] = model.addVar(lb=0.0, name=f"y_{p_id}_{c_id}")

    farm_transport_expr = gp.quicksum(
        (farm_raw_cost[f_id] + farm_to_plant_cost[(f_id, p_id)]) * x[(f_id, p_id)]
        for f_id in farm_ids
        for p_id in plant_ids
    )
    proc_expr = gp.quicksum(
        plant_proc_cost[p_id] * gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids)
        for p_id in plant_ids
    )
    dist_expr = gp.quicksum(
        plant_to_center_cost[(p_id, c_id)] * y[(p_id, c_id)]
        for p_id in plant_ids
        for c_id in center_ids
    )
    model.setObjective(farm_transport_expr + proc_expr + dist_expr, GRB.MINIMIZE)
    for f_id in farm_ids:
        model.addConstr(
            gp.quicksum(x[(f_id, p_id)] for p_id in plant_ids) <= farm_capacity[f_id],
            name=f"Max_Farm_Capacity_Farm_{f_id}"
        )
    for p_id in plant_ids:
        model.addConstr(
            gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids) <= plant_capacity[p_id],
            name=f"Max_Plant_Capacity_Plant_{p_id}"
        )
    for p_id in plant_ids:
        model.addConstr(
            gp.quicksum(y[(p_id, c_id)] for c_id in center_ids) ==
            gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids),
            name=f"PlantFlow_{p_id}"
        )
    for c_id in center_ids:
        model.addConstr(
            gp.quicksum(y[(p_id, c_id)] for p_id in plant_ids) == center_demand[c_id],
            name=f"CenterDem_{c_id}"
        )
        
    model.optimize()
    if model.status == GRB.OPTIMAL:
        print(f"\nOptimal solution found (Quality >= 3 only). Minimum Cost = {model.objVal:,.2f}\n")
        
        farm_plant_data = []
        for (f_id, p_id) in x:
          if x[(f_id, p_id)].X > 1e-6:
            farm_plant_data.append({"Farm": f_id, "Plant": p_id, "Flow (tons)": x[(f_id, p_id)].X})
        
        farm_plant_df = pd.DataFrame(farm_plant_data)
        print("\\nFarm->Plant Flows:\\n", farm_plant_df)

        plant_center_data = []
        for (p_id, c_id) in y:
            if y[(p_id, c_id)].X > 1e-6:
              plant_center_data.append({"Plant": p_id, "Center": c_id, "Flow (tons)": y[(p_id, c_id)].X})

        plant_center_df = pd.DataFrame(plant_center_data)
        print("\\nPlant->Center Flows:\\n", plant_center_df)
    else:
        print(f"Model ended with status {model.status}")

    return model

In [6]:
def solve_bioagri_3pct_limit(farms_csv, processing_csv, centers_csv):
    """
    Builds and solves a multi-stage supply chain model using Gurobi, 
    adding a constraint that each facility can process no more than 3% 
    of the total raw material.
    """
    
    farm_ids, plant_ids, center_ids, farm_capacity, farm_raw_cost, farm_to_plant_cost, plant_capacity, plant_proc_cost, plant_to_center_cost, center_demand, _, _ = read_data(farms_csv, processing_csv, centers_csv)


    model = gp.Model("BioAgri_3pctLimit")

    x = {}
    for f_id in farm_ids:
        for p_id in plant_ids:
            x[(f_id, p_id)] = model.addVar(lb=0.0, name=f"x_{f_id}_{p_id}")
    y = {}
    for p_id in plant_ids:
        for c_id in center_ids:
            y[(p_id, c_id)] = model.addVar(lb=0.0, name=f"y_{p_id}_{c_id}")

    farm_transport_expr = gp.quicksum(
        (farm_raw_cost[f_id] + farm_to_plant_cost[(f_id, p_id)]) * x[(f_id, p_id)]
        for f_id in farm_ids
        for p_id in plant_ids
    )
    proc_expr = gp.quicksum(
        plant_proc_cost[p_id] * gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids)
        for p_id in plant_ids
    )
    dist_expr = gp.quicksum(
        plant_to_center_cost[(p_id, c_id)] * y[(p_id, c_id)]
        for p_id in plant_ids
        for c_id in center_ids
    )
    model.setObjective(farm_transport_expr + proc_expr + dist_expr, GRB.MINIMIZE)
    
    total_demand = sum(center_demand.values())

    alloc = {}
    for f_id in farm_ids:
        for p_id in plant_ids:
            alloc[(f_id, p_id)] = model.addVar(lb=0, ub = 1, name=f"allocation_{f_id}_{p_id}")
    
    for f_id in farm_ids:
           model.addConstr(gp.quicksum(alloc[(f_id,p_id)] for p_id in plant_ids) == 1, name=f"Sum_Allocation_Percentages_{f_id}")
   
    for f_id in farm_ids:
           for p_id in plant_ids:
              model.addConstr(x[(f_id,p_id)] <= alloc[(f_id,p_id)]*farm_capacity[f_id], name=f"Farm_Plant_Capacity_Allocation_{f_id}_{p_id}")
    
    for f_id in farm_ids:
        model.addConstr(
            gp.quicksum(x[(f_id, p_id)] for p_id in plant_ids) <= farm_capacity[f_id],
            name=f"Max_Farm_Capacity_Farm_{f_id}"
        )
    for p_id in plant_ids:
        model.addConstr(
            gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids) <= plant_capacity[p_id],
            name=f"Max_Plant_Capacity_Plant_{p_id}"
        )
    for p_id in plant_ids:
        model.addConstr(
            gp.quicksum(y[(p_id, c_id)] for c_id in center_ids) ==
            gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids),
            name=f"PlantFlow_{p_id}"
        )
    for c_id in center_ids:
        model.addConstr(
            gp.quicksum(y[(p_id, c_id)] for p_id in plant_ids) == center_demand[c_id],
            name=f"CenterDem_{c_id}"
        )
    
    for p_id in plant_ids:
        model.addConstr(
            gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids) <= 0.03 * total_demand,
            name=f"Max3pctPlant_{p_id}"
        )
        
    model.optimize()
    
    if model.status == GRB.OPTIMAL:
      print(f"\nOptimal solution found (3% limit). Minimum Cost = {model.objVal:,.2f}\n")
      for (f_id, p_id) in x:
            if x[(f_id, p_id)].X > 1e-6:
              print(f"From Farm {f_id} -> Plant {p_id}: {x[(f_id, p_id)].X:,.2f} tons")
      for (p_id, c_id) in y:
            if y[(p_id, c_id)].X > 1e-6:
                print(f"From Plant {p_id} -> Center {c_id}: {y[(p_id, c_id)].X:,.2f} tons")
    else:
      print(f"Model ended with status {model.status}")

    return model

In [7]:
def solve_bioagri_50pct_limit_per_center(farms_csv, processing_csv, centers_csv, center_limit=0.5):
    """
    Builds and solves a multi-stage supply chain model using Gurobi, adding a constraint that
    no single plant can supply more than 50% of a given center's demand (risk mitigation).
    """
    farm_ids, plant_ids, center_ids, farm_capacity, farm_raw_cost, farm_to_plant_cost, plant_capacity, plant_proc_cost, plant_to_center_cost, center_demand, _, _ = read_data(farms_csv, processing_csv, centers_csv)

    model = gp.Model("BioAgri_50pctCenter")
    x = {}
    for f_id in farm_ids:
        for p_id in plant_ids:
            x[(f_id, p_id)] = model.addVar(lb=0.0, name=f"x_{f_id}_{p_id}")
    y = {}
    for p_id in plant_ids:
        for c_id in center_ids:
            y[(p_id, c_id)] = model.addVar(lb=0.0, name=f"y_{p_id}_{c_id}")
    
    farm_transport_expr = gp.quicksum(
        (farm_raw_cost[f_id] + farm_to_plant_cost[(f_id, p_id)]) * x[(f_id, p_id)]
        for f_id in farm_ids
        for p_id in plant_ids
    )
    proc_expr = gp.quicksum(
        plant_proc_cost[p_id] * gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids)
        for p_id in plant_ids
    )
    dist_expr = gp.quicksum(
        plant_to_center_cost[(p_id, c_id)] * y[(p_id, c_id)]
        for p_id in plant_ids
        for c_id in center_ids
    )

    model.setObjective(farm_transport_expr + proc_expr + dist_expr, GRB.MINIMIZE)
   
    for f_id in farm_ids:
        model.addConstr(
            gp.quicksum(x[(f_id, p_id)] for p_id in plant_ids) <= farm_capacity[f_id],
            name=f"Max_Farm_Capacity_Farm_{f_id}"
        )
    for p_id in plant_ids:
        model.addConstr(
            gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids) <= plant_capacity[p_id],
            name=f"Max_Plant_Capacity_Plant_{p_id}"
        )
    for p_id in plant_ids:
        model.addConstr(
            gp.quicksum(y[(p_id, c_id)] for c_id in center_ids) ==
            gp.quicksum(x[(f_id, p_id)] for f_id in farm_ids),
            name=f"Flow_Conservation_Plant_{p_id}"
        )
    for c_id in center_ids:
        model.addConstr(
            gp.quicksum(y[(p_id, c_id)] for p_id in plant_ids) == center_demand[c_id],
            name=f"Center_Demand_Center_{c_id}"
        )
    
    for c_id in center_ids:
        demand_c = center_demand[c_id]
        for p_id in plant_ids:
            model.addConstr(
                y[(p_id, c_id)] <=  center_limit * demand_c,
                name=f"Max_50pct_Plant_{p_id}_To_Center_{c_id}"
            )
    model.optimize()
    
    if model.status == GRB.OPTIMAL:
        print(f"\nOptimal solution found (50% center limit). Minimum Cost = {model.objVal:,.2f}\n")
        
        farm_plant_data = []
        for (f_id, p_id) in x:
          if x[(f_id, p_id)].X > 1e-6:
            farm_plant_data.append({"Farm": f_id, "Plant": p_id, "Flow (tons)": x[(f_id, p_id)].X})
        
        farm_plant_df = pd.DataFrame(farm_plant_data)
        print("\\nFarm->Plant Flows:\\n", farm_plant_df)
        
        plant_center_data = []
        for (p_id, c_id) in y:
            if y[(p_id, c_id)].X > 1e-6:
              plant_center_data.append({"Plant": p_id, "Center": c_id, "Flow (tons)": y[(p_id, c_id)].X})

        plant_center_df = pd.DataFrame(plant_center_data)
        print("\\nPlant->Center Flows:\\n", plant_center_df)
    else:
        print(f"Model ended with status {model.status}")

    return model

In [8]:
if __name__ == "__main__":
    
    farms_csv = "c:/MBAN- Schulich/MBAN- Sem 3/Models & Applications in Op Research/Assignment 1/farms.csv"
    processing_csv = "c:/MBAN- Schulich/MBAN- Sem 3/Models & Applications in Op Research/Assignment 1/processing.csv"
    centers_csv = "c:/MBAN- Schulich/MBAN- Sem 3/Models & Applications in Op Research/Assignment 1/centers.csv"

    # (b) Solve base Model
    print("Base Model:\n")
    model_b = solve_bioagri(farms_csv, processing_csv, centers_csv)
    
    # (c) Solve same-region model
    print("\nSame Region Model:\n")
    model_c = solve_bioagri_same_region(farms_csv, processing_csv, centers_csv)

    # (d) Solve high-quality model
    print("\nHigh Quality Model:\n")
    model_d = solve_bioagri_high_quality(farms_csv, processing_csv, centers_csv)
    
     # (e) solve plant constraint
    print("\n3% plant constraint Model:\n")
    model_e_1 = solve_bioagri_3pct_limit(farms_csv, processing_csv, centers_csv)

    # (e) solve 50% center constraint
    print("\n50% Center Limit Model:\n")
    model_e_2 = solve_bioagri_50pct_limit_per_center(farms_csv, processing_csv, centers_csv, center_limit=0.5)

Base Model:

Set parameter Username
Set parameter LicenseID to value 2610005
Academic license - for non-commercial use only - expires 2026-01-14
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 387 rows, 6318 columns and 17118 nonzeros
Model fingerprint: 0x14917975
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 3e+04]
Presolve time: 0.01s
Presolved: 387 rows, 6318 columns, 17118 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.1171568e+04   2.902000e+04   0.000000e+00      0s
     370    2.2970900e+06   0.000000e+00   0.000000e+00      0s

Solved in 370 iterations and 0.04 seconds (0.02 work units)
Optimal objective  2.297089973e+0